In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [27]:
wk_dir = '../../data'

In [21]:
train_df['date'] = train_df['date'] + pd.offsets.DateOffset(years=4)

In [22]:
new_train_df = train_df.loc[(train_df['store'] == 1) | (train_df['store'] == 2)]
new_train_df = new_train_df.loc[(new_train_df['item'] == 1) | (new_train_df['item'] == 2) | (new_train_df['item'] == 3)]

In [29]:
def create_data(data, add_yr):
    data['date'] = data['date'] + pd.offsets.DateOffset(years=add_yr)
    data = data.loc[(data['store'] == 1) | (data['store'] == 2)]
    data = data.loc[(data['item'] == 1) | (data['item'] == 2) | (data['item'] == 3)]
    data.rename(columns={'store': 'ticket type', 'item': 'ticket class'}, inplace=True)
    data['ticket type'].replace({1: 'return', 2: 'one-way'}, inplace=True)
    data['ticket class'].replace({1: 'first', 2: 'business', 3: 'economy'}, inplace=True)
    
    return data

In [41]:
train_df = pd.read_csv(f'{wk_dir}/train.csv', index_col=False, parse_dates=['date'])
test_df = pd.read_csv(f'{wk_dir}/test.csv', index_col=False, parse_dates=['date'])

train_df = create_data(train_df, 4)
test_df = create_data(test_df, 4)

test_df.drop(columns=['id'], inplace=True)
test_df = test_df.loc[test_df['date'].dt.month == 1]

train_df.to_csv(f'{wk_dir}/airline_flights.csv', index=False)
train_df.to_csv(f'{wk_dir}/airline_flights_future.csv', index=False)

In [40]:
test_df['date'].dt.year.unique()
test_df = test_df.loc[test_df['date'].dt.month == 1]
test_df['date'].dt.month.unique()

array([1])

In [24]:
new_train_df.rename(columns={'store': 'ticket type', 'item': 'ticket class'}, inplace=True)
new_train_df['ticket type'].replace({1: 'return', 2: 'one-way'}, inplace=True)
new_train_df['ticket class'].replace({1: 'first', 2: 'business', 3: 'economy'}, inplace=True)
new_train_df.head()

,date,ticket type,ticket class,sales
0,2017-01-01,return,first,13
1,2017-01-02,return,first,11
2,2017-01-03,return,first,14
3,2017-01-04,return,first,13
4,2017-01-05,return,first,10


In [25]:
new_train_df.describe()

,sales
count,10956.000000
mean,42.801844
std,22.557771
min,4.000000
25%,25.000000
50%,38.000000
75%,56.000000
max,150.000000


In [26]:
train_df['date'].max()

Timestamp('2021-12-31 00:00:00')

In [18]:
new_train_df.to_csv('../../data/demand_data.csv', index=False)

In [ ]:
y_df = train_df['sales']
train_df = train_df[['store', 'item']]

In [ ]:
tt_x, dev_x, tt_y, dev_y = train_test_split(train_df, y_df, test_size=0.3, random_state=2022)

In [ ]:
model = xgb.XGBRegressor(max_depth=7, num_leaves=200)

In [ ]:
model.fit(tt_x, tt_y)

In [ ]:
### save model
import pickle as pkl

filename = 'xgb_model_2.pkl'
pkl.dump(model, open(f'../../models/{filename}', "wb"))

In [ ]:
### load model and use to predict
model_loaded = pkl.load(open(f'../../models/{filename}', "rb"))

In [ ]:
predictions = model_loaded.predict(dev_x)

In [ ]:
predictions

In [ ]:
#### ticket type: return ticket, one way
#### ticket class first class, business class, economy